# Figures for ESANN Paper

Might be a good idea to work on the draft of the paper before actually making the figures but I might have to get some solid figure first and then make my story around these figures.

First lets get the data needed in here.

In [ ]:
import strax 
import straxen 
import numpy as np
import matplotlib.pyplot as plt
import cutax
import pandas as pd
import seaborn as sns

from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
import matplotlib.image as image
from matplotlib import cm
from matplotlib.lines import Line2D

In [ ]:
def assign_labels(data, ref_img, xdim, ydim, cut_out):
    '''This functions takes in the data and classifications based on an image gives the
    unique labels as well as the data set bacl with the new classification
    PS this version only takes in S1s and S2s and ignores unclassified samples, 
    another version will be made to deal with the unclassified samples
    
    data: can be either peaks or peak_basics
    ref_img: will be the image extracted from the SOM classification of each data point
    xdim: width of the image cube
    ydim: height of the image cube
    cut_out: removes the n last digits of the image vector if necesarry'''
    from PIL import Image
    data_new = data
    img = Image.open(ref_img)
    imgGray = img.convert('L')
    #imgGray2.save('/home/luissanchez25/im_kr83_real__30x30_2lbl.0.rmpmap.png')
    img_color = np.array(img) #still in the x,y,3 format
    img_color_2d = img_color.reshape((xdim*ydim,3))
    label = -1 * np.ones(img_color.shape[:-1])
    colorp = np.unique(img_color_2d, axis = 0)
    for i, color in enumerate(colorp):  # argwhere
        label[np.all((img_color == color), axis = 2)] = i #assignes each color a number
    label_vec = label.reshape((xdim*ydim))
    if cut_out != 0:
        label_vec_nonzero = label_vec[:-cut_out]
    elif cut_out == 0:
        label_vec_nonzero = label_vec
    #s2_data = data[data['type'] == 2]
    #s1_data = data[data['type'] == 1]
    #som_class_peaks = np.concatenate((s2_data,s1_data)) #note had to reverse s2,s1 to s1,s2
    data_new['type'] = label_vec_nonzero
    
    return colorp, data_new

In [ ]:
def change_type(data, cls_array):
    #makes sure you give a label to each data set
    data_s1s2 = data.copy()
    assert len(cls_array) == len(np.unique(data['type']))
    for i in np.arange(len(np.unique(data_s1s2['type']))):
        data_s1s2['type'] = np.where(data_s1s2['type'] != i, data_s1s2['type'], cls_array[i])
        
    return data_s1s2

In [ ]:
st = cutax.contexts.xenonnt_offline()

In [ ]:
krpb_wrep = np.load('./saved_data/clean_kr_cal_pb.npz')['arr_0']

In [ ]:
_,krpb_data_indecies = np.unique(krpb_wrep, return_index=True)

In [ ]:
krpb_non_repeat = krpb_wrep[np.sort(krpb_data_indecies)]

In [ ]:
color_daa, som_cls_dec_area_aft = assign_labels(krpb_non_repeat, './SOM_results/im_kr_034373_decile_area_aft_40x40.0.rmpmap.png', 152, 991, 1)

In [ ]:
color_daa2, som_cls_dec_area_aft2 = assign_labels(krpb_non_repeat, './SOM_results/im_kr034373_decile_area_aft_40x40_v2.0.rmpmap.png', 152, 991, 1)

In [ ]:
color_daa2[19] = np.array([0,0,0])

In [ ]:
fig = plt.figure(figsize=(8,6))
for kind in np.arange(len(np.unique(som_cls_dec_area_aft2['type']))):
    plt.scatter(som_cls_dec_area_aft2['area'][som_cls_dec_area_aft2['type'] == kind], 
                som_cls_dec_area_aft2['rise_time'][som_cls_dec_area_aft2['type'] == kind], 
                s=0.5, alpha = 0.5, color = color_daa2[kind]/255)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area')
plt.ylabel('Rise Time')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=4, figsize=(18, 20))

num = 0
for i in np.arange(5):
    for j in np.arange(4):
        ax[i,j].scatter(som_cls_dec_area_aft2['area'][som_cls_dec_area_aft2['type'] == num], 
                        som_cls_dec_area_aft2['rise_time'][som_cls_dec_area_aft2['type'] == num], 
                        s=0.1, color = color_daa2[num]/255, alpha = 1)
        ax[i,j].set_xscale('log')
        ax[i,j].set_yscale('log')
        ax[i,j].set_xlim(1,1000000)
        ax[i,j].set_ylim(1,100000)
        num = num + 1

In [ ]:
S1_S2_list = np.array([1,1,2,2,
                       2,1,2,2,
                       2,1,1,2,
                       2,1,1,2,
                       2,1,2,1])

In [ ]:
som_cls_s1s2_dec_area_aft2 = som_cls_dec_area_aft2

In [ ]:
data_s1s2 = change_type(som_cls_s1s2_dec_area_aft2, S1_S2_list)

In [ ]:
fig = plt.figure(figsize=(6,4.5))

plt.scatter(data_s1s2['area'][data_s1s2['type'] == 1], 
            data_s1s2['rise_time'][data_s1s2['type'] == 1], 
            s=0.1, alpha = 0.5, color = 'blue', label = 'SOM S1')
plt.scatter(data_s1s2['area'][data_s1s2['type'] == 2], 
            data_s1s2['rise_time'][data_s1s2['type'] == 2], 
            s=0.1, alpha = 0.5, color = 'green', label = 'SOM S2')
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise Time [ns]')
plt.xlim(1,1000000)
plt.ylim(1,100000)
plt.legend(markerscale=8)

In [ ]:
d = {'Area [PE]': krpb_non_repeat['area'], 
     'Rise-Time [ns]': krpb_non_repeat['rise_time'],
     'kind': krpb_non_repeat['type']}

In [ ]:
df = pd.DataFrame(data=d)

In [ ]:
fig, ax = plt.subplots()
data = krpb_non_repeat[krpb_non_repeat['type'] == 1]
data_list = {'Area [PE]': data['area'], 
 'Rise-Time [ns]': data['rise_time'],
 'kind': data['type']}
data_df = pd.DataFrame(data=data_list)

#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", hue="kind",
                 levels=2, thresh=0.05, fill = True, alpha=.4,
                 color=custom_palette2[1], log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)

In [ ]:
fig, ax = plt.subplots()
for i in np.arange(len(custom_palette2)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == i ]
    data_list = {'Area [PE]': data['area'], 
     'Rise-Time [ns]': data['rise_time'],
     'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    #sns.set_palette(custom_palette)
    ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", # hue="kind",
                     levels=2, thresh=0.1, fill = True, alpha=.4,
                     color=custom_palette2[i], log_scale = True, ax = ax)

ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)


In [ ]:
list_of_values = [1,3,4,8,12,13,16,17]
fig, ax = plt.subplots()
for i in np.arange(len(list_of_values)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == list_of_values[i] ]
    data_list = {'Area [PE]': data['area'], 
     'Rise-Time [ns]': data['rise_time'],
     'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    #sns.set_palette(custom_palette)
    ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", # hue="kind",
                     levels=2, thresh=0.05, fill = True, alpha=.4,
                     color=custom_palette2[list_of_values[i]], log_scale = True, ax = ax)

ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)

In [ ]:
list_of_values = [1,3,4,8,12,13,16,17]
fig, ax = plt.subplots()
for i in np.arange(len(list_of_values)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == list_of_values[i] ]
    data_list = {'Area [PE]': data['area'], 
     'Rise-Time [ns]': data['rise_time'],
     'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    #sns.set_palette(custom_palette)
    ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", # hue="kind",
                     levels=2, thresh=0.05, fill = True, alpha=.8,
                     color=custom_palette2[list_of_values[i]], log_scale = True, ax = ax)

ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()
for i in np.arange(len(list_of_values)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == list_of_values[i] ]
    data_list = {'Area [PE]': data['area'], 
     'Rise-Time [ns]': data['rise_time'],
     'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    #sns.set_palette(custom_palette)
    ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", # hue="kind",
                     levels=2, thresh=0.05, fill = True, alpha=.8,
                     color=custom_palette2[list_of_values[i]], log_scale = True, ax = ax)

ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()
for i in np.arange(len(list_of_values)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == list_of_values[i] ]
    data_list = {'Area [PE]': data['area'], 
     'Rise-Time [ns]': data['rise_time'],
     'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    #sns.set_palette(custom_palette)
    ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", # hue="kind",
                     levels=2, thresh=0.03, fill = True, alpha=.8,
                     color=custom_palette2[list_of_values[i]], log_scale = True, ax = ax)

ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)

In [ ]:
custom_palette2 = ['#1d267d', '#2b3be2', '#216916', '#4cf931', 
                   '#18b000', '#19aff0', '#5b676c', '#71560e',
                   '#6e38db', '#5bf5ff', '#b595f9', '#b2b2b2', 
                   '#cbfab3', '#b5b200', '#ad8b7b', '#ddd726', 
                   '#f7a102', '#f70a02', '#ff8a80', '#000000']

In [ ]:
fig, ax = plt.subplots()
custom_palette = sns.color_palette("husl", 4)
#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=df, x="Area [PE]", y="Rise-Time [ns]", hue="kind", hue_label={"S1": "1", "S2": "2"},
            palette=custom_palette2, log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots()
custom_palette = sns.color_palette("husl", 4)
#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=df, x="Area [PE]", y="Rise-Time [ns]", hue="kind", hue_label={"S1": "1", "S2": "2"},
                 linewidths=0.8,
                 palette=custom_palette2, log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.legend(loc='upper left')

In [ ]:
# , linewidths=0.5
fig, ax = plt.subplots()
custom_palette = sns.color_palette("husl", 4)
#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=df, x="Area [PE]", y="Rise-Time [ns]", hue="kind", hue_label={"S1": "1", "S2": "2"},
                 linewidths=0.5,
                 palette=custom_palette2, log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots()
custom_palette = sns.color_palette("husl", 4)
#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=df, x="Area [PE]", y="Rise-Time [ns]", hue="kind", hue_label={"S1": "1", "S2": "2"},
                 levels=5, thresh=0.01,
                 palette=custom_palette2, log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots()
custom_palette = sns.color_palette("husl", 4)
#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=df, x="Area [PE]", y="Rise-Time [ns]", hue="kind", hue_label={"S1": "1", "S2": "2"},
                 levels=5, thresh=0.03, linewidths=0.5,
                 palette=custom_palette2, log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots()
custom_palette = sns.color_palette("husl", 4)
#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=df, x="Area [PE]", y="Rise-Time [ns]", hue="kind", hue_label={"S1": "1", "S2": "2"},
                 linewidths=0.8,
                 palette=custom_palette2, log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots()
custom_palette = sns.color_palette("husl", 4)
#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=df, x="Area [PE]", y="Rise-Time [ns]", hue="kind", hue_label={"S1": "1", "S2": "2"},
                 levels=5, thresh=0.03, linewidths=0.8,
                 palette=custom_palette2, log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots()
custom_palette = sns.color_palette("husl", 4)
#sns.set_palette(custom_palette)
ax = sns.kdeplot(data=df, x="Area [PE]", y="Rise-Time [ns]", hue="kind", hue_label={"S1": "1", "S2": "2"},
                 levels=8, thresh=0.03, linewidths=0.8,
                 palette=custom_palette2, log_scale = True, ax = ax)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.legend(loc='upper left')

In [ ]:
# levels=10, thresh=0.05,

In [ ]:
fig = plt.figure(figsize=(8,6))
for kind in np.arange(len(np.unique(som_cls_dec_area_aft2['type']))):
    plt.scatter(som_cls_dec_area_aft2['area'][som_cls_dec_area_aft2['type'] == kind], 
                som_cls_dec_area_aft2['rise_time'][som_cls_dec_area_aft2['type'] == kind], 
                s=0.5, alpha = 0.5, color = color_daa2[kind]/255)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('Rise-Time [ns]')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
fig = plt.figure(figsize=(8,6))
for kind in np.arange(len(np.unique(som_cls_dec_area_aft2['type']))):
    plt.scatter(som_cls_dec_area_aft2['area'][som_cls_dec_area_aft2['type'] == kind], 
                som_cls_dec_area_aft2['range_50p_area'][som_cls_dec_area_aft2['type'] == kind], 
                s=0.5, alpha = 0.5, color = color_daa2[kind]/255)
#plt.scatter(kr_pb_array['area'][kr_pb_array['type'] == 1], kr_pb_array['rise_time'][kr_pb_array['type'] == 1], s=0.1, color = 'blue')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area [PE]')
plt.ylabel('50% width [ns]')
plt.xlim(1,1000000)
plt.ylim(1,100000)

In [ ]:
som_cls_dec_area_aft2.dtype

In [ ]:
custom_palette2 = ['#1d267d', '#2b3be2', '#216916', '#4cf931', 
                   '#18b000', '#19aff0', '#5b676c', '#71560e',
                   '#6e38db', '#5bf5ff', '#b595f9', '#b2b2b2', 
                   '#cbfab3', '#b5b200', '#ad8b7b', '#ddd726', 
                   '#f7a102', '#f70a02', '#ff8a80', '#000000']

In [ ]:
file = './logos/XE_preliminary_horizontal.png'

In [ ]:
logo = image.imread(file)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()
for i in np.arange(len(list_of_values)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == list_of_values[i] ]
    data_list = {'Area [PE]': data['area'], 
     'Rise-Time [ns]': data['rise_time'],
     'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    #sns.set_palette(custom_palette)
    ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", # hue="kind",
                     levels=2, thresh=0.05, fill = True, alpha=.8,
                     color=custom_palette2[list_of_values[i]], log_scale = True, ax = ax)
imagebox = OffsetImage(logo, zoom = 0.05)
#Annotation box for solar pv logo
#Container for the imagebox referring to a specific position *xy*.
ab = AnnotationBbox(imagebox, (80000, 100),
                    frameon = False)
ax.add_artist(ab)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
plt.savefig('./figures/SOM_rt_v_area_95%_contours.png', dpi = 400)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()
for i in np.arange(len(list_of_values)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == list_of_values[i] ]
    data_list = {'Area [PE]': data['area'], 
     'Rise-Time [ns]': data['rise_time'],
     'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    #sns.set_palette(custom_palette)
    ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", # hue="kind",
                     levels=5, thresh=0.05,  alpha=.8,
                     color=custom_palette2[list_of_values[i]], log_scale = True, ax = ax)
imagebox = OffsetImage(logo, zoom = 0.05)
#Annotation box for solar pv logo
#Container for the imagebox referring to a specific position *xy*.
ab = AnnotationBbox(imagebox, (80000, 100),
                    frameon = False)
ax.add_artist(ab)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()
for i in np.arange(len(list_of_values)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == list_of_values[i] ]
    data_list = {'Area [PE]': data['area'], 
     'Rise-Time [ns]': data['rise_time'],
     'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    #sns.set_palette(custom_palette)
    ax = sns.kdeplot(data=data_df, x="Area [PE]", y="Rise-Time [ns]", # hue="kind",
                     levels=2, thresh=0.03, fill = True, alpha=.8,
                     color=custom_palette2[list_of_values[i]], log_scale = True, ax = ax)
imagebox = OffsetImage(logo, zoom = 0.05)
#Annotation box for solar pv logo
#Container for the imagebox referring to a specific position *xy*.
ab = AnnotationBbox(imagebox, (80000, 100),
                    frameon = False)
ax.add_artist(ab)
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
plt.savefig('./figures/SOM_rt_v_area_97%_contours.png', dpi = 400)

In [ ]:
logo_left = 10
logo_bottom = 1000
logo_width = 100000
logo_height = 10000

In [ ]:
fig, ax = plt.subplots()
ax.imshow(logo, aspect='auto', extent=(logo_left, logo_left+logo_width, logo_bottom, logo_bottom+logo_height), zorder=-1)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()

for i in np.arange(len(list_of_values)):
    data = krpb_non_repeat[krpb_non_repeat['type'] == list_of_values[i]]
    data_list = {'Area [PE]': data['area'], 
                 'Rise-Time [ns]': data['rise_time'],
                 'kind': data['type']}
    data_df = pd.DataFrame(data=data_list)
    
    plt.scatter(data_df["Area [PE]"], data_df["Rise-Time [ns]"],
                alpha=0.8, color=custom_palette2[list_of_values[i]], s=0.5)

# Adjust logo position and size
imagebox = OffsetImage(logo, zoom = 0.05)
#Annotation box for solar pv logo
#Container for the imagebox referring to a specific position *xy*.
ab = AnnotationBbox(imagebox, (80000, 100),
                    frameon = False)
ax.add_artist(ab)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.set_xlabel('Area [PE]')
ax.set_ylabel('Rise Time [ns]')
plt.savefig('./figures/SOM_rt_v_area_scatter_plot.png', dpi = 400)

In [ ]:
color_list=[]
color_array = np.zeros(len(krpb_non_repeat['type']))
for i in np.arange(len(krpb_non_repeat['type'])):
    if krpb_non_repeat['type'][i] == 1:
        #color_array[i] = custom_palette2[1]
        color_list = np.append(color_list, custom_palette2[1])
    if krpb_non_repeat['type'][i] == 3:
        #color_array[i] = custom_palette2[3]
        color_list = np.append(color_list, custom_palette2[3])
    if krpb_non_repeat['type'][i] == 8:
        #color_array[i] = custom_palette2[8]
        color_list = np.append(color_list, custom_palette2[8])
    if krpb_non_repeat['type'][i] == 12:
        #color_array[i] = custom_palette2[12]
        color_list = np.append(color_list, custom_palette2[12])
    if krpb_non_repeat['type'][i] == 13:
        #color_array[i] = custom_palette2[13]
        color_list = np.append(color_list, custom_palette2[13])
    if krpb_non_repeat['type'][i] == 16:
        #color_array[i] = custom_palette2[16]
        color_list = np.append(color_list, custom_palette2[16])
    if krpb_non_repeat['type'][i] == 17:
        #color_array[i] = custom_palette2[17]
        color_list = np.append(color_list, custom_palette2[17])
    

In [ ]:
custom_palette2[17].dtype()

In [ ]:
col = np.where(krpb_non_repeat['type'] == 1,custom_palette2[1],
              krpb_non_repeat['type'] == 3,custom_palette2[3],
              krpb_non_repeat['type'] == 8,custom_palette2[8],
              krpb_non_repeat['type'] == 12,custom_palette2[12],
              krpb_non_repeat['type'] == 13,custom_palette2[13],
              krpb_non_repeat['type'] == 16,custom_palette2[16],
              krpb_non_repeat['type'] == 17,custom_palette2[17])

In [ ]:
col = np.where(krpb_non_repeat['type'] == 1,custom_palette2[1])

In [ ]:
custom_palette2[1]

In [ ]:
data['area']

In [ ]:
len(krpb_non_repeat)

In [ ]:
kr_cut_data = krpb_non_repeat.copy()

In [ ]:
mask =  (kr_cut_data["type"] == 1) | (kr_cut_data["type"] == 3) | (kr_cut_data["type"] == 8) | (kr_cut_data["type"] == 12) | (kr_cut_data["type"] == 13) | (kr_cut_data["type"] == 16) | (kr_cut_data["type"] == 17)

In [ ]:
kr_cut_data = kr_cut_data[mask]

In [ ]:
len(kr_cut_data)

In [ ]:
len(color_list)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()

for i in np.arange(len(kr_cut_data)):
    
    plt.scatter(kr_cut_data['area'][i], kr_cut_data['rise_time'][i],
                alpha=0.8, color=color_list[i], s=2)

# Adjust logo position and size
imagebox = OffsetImage(logo, zoom = 0.05)
#Annotation box for solar pv logo
#Container for the imagebox referring to a specific position *xy*.
ab = AnnotationBbox(imagebox, (80000, 100),
                    frameon = False)
ax.add_artist(ab)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()
    
plt.scatter(kr_cut_data['area'], kr_cut_data['rise_time'],
            alpha=0.8, color=color_list, s=0.5)

# Adjust logo position and size
imagebox = OffsetImage(logo, zoom = 0.05)
#Annotation box for solar pv logo
#Container for the imagebox referring to a specific position *xy*.
ab = AnnotationBbox(imagebox, (80000, 100),
                    frameon = False)
ax.add_artist(ab)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)

In [ ]:
random = np.random.choice(len(kr_cut_data), len(kr_cut_data), replace=False)

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()
    
plt.scatter(kr_cut_data['area'][random], kr_cut_data['rise_time'][random],
            alpha=0.8, color=color_list[random], s=0.5)

# Adjust logo position and size
imagebox = OffsetImage(logo, zoom = 0.05)
#Annotation box for solar pv logo
#Container for the imagebox referring to a specific position *xy*.
ab = AnnotationBbox(imagebox, (80000, 100),
                    frameon = False)
ax.add_artist(ab)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.set_xlabel('Area [PE]')
ax.set_ylabel('Rise-Time [ns]')

In [ ]:
list_of_values = [1,3,8,12,13,16,17]
fig, ax = plt.subplots()
    
plt.scatter(kr_cut_data['area'][random], kr_cut_data['rise_time'][random],
            alpha=0.4, color=color_list[random], s=0.5)

# Adjust logo position and size
imagebox = OffsetImage(logo, zoom = 0.05)
#Annotation box for solar pv logo
#Container for the imagebox referring to a specific position *xy*.
ab = AnnotationBbox(imagebox, (80000, 100),
                    frameon = False)
ax.add_artist(ab)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1, 10000000)
ax.set_ylim(10, 100000)
ax.set_xlabel('Area [PE]')
ax.set_ylabel('Rise-Time [ns]')

In [ ]:
kr_cut_data['area']